In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("creditcard.csv")

In [3]:
data.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0


In [4]:
data.shape

(284807, 31)

In [5]:
data["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [6]:
X = data.drop("Class", axis = 1)
y = data["Class"]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [9]:
log_class = LogisticRegression()
grid = {"C": 10.0 ** np.arange(-2, 3), "penalty" : ['l1', 'l2']}
cv = KFold(n_splits = 5, random_state = None, shuffle = False) 

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7)

In [11]:
clf = GridSearchCV(log_class, grid, cv = cv, n_jobs = -1, scoring = "f1_macro")
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-pack

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [12]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[85247    41]
 [   47   108]]
0.9989700736163291
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85288
           1       0.72      0.70      0.71       155

    accuracy                           1.00     85443
   macro avg       0.86      0.85      0.86     85443
weighted avg       1.00      1.00      1.00     85443



In [13]:
y_train.value_counts()

0    199027
1       337
Name: Class, dtype: int64

In [14]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(class_weight = dict({0: 1, 1: 100}))
classifier.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [15]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[85280     8]
 [   26   129]]
0.999602073897218
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85288
           1       0.94      0.83      0.88       155

    accuracy                           1.00     85443
   macro avg       0.97      0.92      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [16]:
from collections import Counter
Counter(y_train)

Counter({0: 199027, 1: 337})

In [21]:
from imblearn.under_sampling import RandomUnderSampler
ns = RandomUnderSampler(0.8)
X_train_ns, y_train_ns = ns.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

The number of classes before fit Counter({0: 199027, 1: 337})
The number of classes after fit Counter({0: 421, 1: 337})


/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:591: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


In [22]:
classifier = RandomForestClassifier()
classifier.fit(X_train_ns, y_train_ns)

RandomForestClassifier()

In [23]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[82356  2932]
 [    6   149]]
0.9656145032360756
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     85288
           1       0.05      0.96      0.09       155

    accuracy                           0.97     85443
   macro avg       0.52      0.96      0.54     85443
weighted avg       1.00      0.97      0.98     85443



In [24]:
from imblearn.over_sampling import RandomOverSampler

In [28]:
os = RandomOverSampler(sampling_strategy = 0.75)
X_train_ns, y_train_ns = os.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

The number of classes before fit Counter({0: 199027, 1: 337})
The number of classes after fit Counter({0: 199027, 1: 149270})


In [30]:
classifier.fit(X_train_ns, y_train_ns)

RandomForestClassifier()

In [32]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[85281     7]
 [   24   131]]
0.9996371850239341
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85288
           1       0.95      0.85      0.89       155

    accuracy                           1.00     85443
   macro avg       0.97      0.92      0.95     85443
weighted avg       1.00      1.00      1.00     85443

